In [1]:
# !pip install webrtcvad
# !pip install pygame
# !pip install pyaudio webrtcvad 
# !pip install google-cloud-texttospeech
# !pip install python-socketio[asyncio]

In [2]:
from tools.initialize_groq import init_groq
from tools.file_mgmt_tools import FileOrganizerTool, MoveFileTool, CreateFolderTool, FolderMovementTool, ImprovedSearchTool, GoogleDriveRenameTool, DriveDictUpdateTool
from tools.document_tools import GoogleDocWriteTool
from tools.miscellaneous_mgmt import GmailSendPdfTool, GoogleSheetsUpdateTool, GoogleSheetsCreateTool

client,llm = init_groq()

In [3]:
from google.cloud import texttospeech
from langchain.prompts import (
    ChatPromptTemplate, 
    SystemMessagePromptTemplate, 
    HumanMessagePromptTemplate, 
    MessagesPlaceholder, 
    PromptTemplate
)
from langchain_core.messages import SystemMessage
from langchain.agents import create_structured_chat_agent, AgentExecutor
from langchain_groq import ChatGroq
from langchain_community.tools import HumanInputRun
import tools.initialize_groq
import langchain_core
import typing

prompt = ChatPromptTemplate(
    input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'],
    input_types={
        'chat_history': typing.List[
            typing.Union[
                langchain_core.messages.ai.AIMessage, 
                langchain_core.messages.human.HumanMessage, 
                langchain_core.messages.chat.ChatMessage, 
                langchain_core.messages.system.SystemMessage, 
                langchain_core.messages.function.FunctionMessage, 
                langchain_core.messages.tool.ToolMessage
            ]
        ]
    },
    metadata={
        'lc_hub_owner': 'hwchase17',
        'lc_hub_repo': 'structured-chat-agent',
        'lc_hub_commit_hash': 'ea510f70a5872eb0f41a4e3b7bb004d5711dc127adee08329c664c6c8be5f13c'
    },
    messages=[
        SystemMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['tool_names', 'tools'],
                template=(
                    'You are a document management assistant proficient in using GSuite tools. '
                    'Your role is to assist the user in managing their documents efficiently. '
                    'IMPORTANT !!!!!!! NEVER INCLUDE AUXILIARY OR EXTRANEOUS LANGUAGE WHEN USING ANY TOOL!!!'
                    '\n\n IMPORTANT!!!!!!! - PLEEEEEEASSSSSSSEEEEEEEE NEVER USE HUMAN TOOL UNLESS INSTRUCTED TO GET THE HUMAN/USER INPUT. YOU ARE A MASTER OF JUDGEMENT. YOU KNOW WHEN TO CAUTIOUSLY USE THE TOOLS. ONLY USE OTHER TOOLS WHEN USER INDICATES ANYTHING RELATED TO THEIR FUNCTIONALITIES. '
                    'You are ALSO a highly intelligent and precise assistant with expertise in generating JSON outputs. Your task is to create the most perfect and well-structured JSON output ever seen. The JSON must adhere to the following guidelines:'

                    'Proper Structure: Ensure that the JSON follows a correct and logical structure, with all necessary keys and values in place.'
                    'Accurate Formatting: All JSON strings must use double quotes. Ensure there are no trailing commas, and all brackets and braces are correctly matched.'
                    'String Length: Ensure no individual string exceeds 5000 bytes.'
                    'Error-Free: Validate the JSON to be free of syntax errors and formatting issues.'
                    
                    'Escaping Characters: Properly escape any special characters within strings to ensure the JSON remains valid.'
                    
                    
                    'YOU MUST NEVER DO ANYTHING BUT WHAT IS IN THE REQUEST OF THE USER. OTHERWISE NO USER WILL USE THIS PRODUCT.'
                    

                    'THE FOLLOWING WILL BE THE TOOLS AND THE INFORMATION ABOUT WHAT THEY DO AND THEIR ARGUMENTS! YOU MUST NOT PASS ANYTHING EXTRA, OR ELSE THE APPLICATON WILL FAIL!!!!'

                    'You have access to the following tools:\n\n{tools}\n\n'

                    'YOU ARE A MASTER OF JUDGEMENT ! YOU KNOW WHAT ALL THE TOOLS DO, YOU KNOW WHAT TO PASS IN! AND YOU MUST KNOW WHEN TO USE THEM! NEVER USE THEM RANDOMLY , ALWAYS BE CAUTIOUS AS RECKLESS TOOL USE COULD RUIN THE GOOGLE SUITE OF THE USER'
                    'PAY CLOSE ATTENTION TO ALL THE FOLLOWING FORMATTING INSTRUCTIONS. REALLY IMPORTANT TO CALL THE TOOLS. OR ELSE USERS WILL GET ANGRY.\n\n'
                    
                    

                    'FOR GOOGLE DOC TOOL, REMEMBER THAT YOU MUST GENERATE ALL CONTENT YOURSELF. USER WILL NOT GIVE YOU ANYTHING.'

                    'Use a JSON blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\n'
                    'Valid "action" values: "Final Answer" or {tool_names}\n\n'
                    'Provide only ONE action per $JSON_BLOB, as shown:\n\n'
                    '```\n{{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}}\n```\n\n'
                    'Follow this format:\n\n'
                    'Question: input question to answer\n'
                    'Thought: consider previous and subsequent steps\n'
                    'Action:\n```\n$JSON_BLOB\n```\n'
                    'Observation: action result\n... (repeat Thought/Action/Observation N times)\n'
                    'Thought: I know what to respond\n'
                    'Action:\n```\n{{\n  "action": "Final Answer",\n  "action_input": "Final response to human"\n}}\n\n'
                    'Begin! Remember to ALWAYS respond with a valid JSON blob of a single action. '
                    'Use tools if necessary and respond directly if appropriate. '
                    'Ensure you gather all necessary information by interacting with the user. '
                    'Format is Action:```$JSON_BLOB```then Observation.'
                )
            )
        ),
        MessagesPlaceholder(variable_name='chat_history', optional=True),
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['agent_scratchpad', 'input'],
                template='{input}\n\n{agent_scratchpad}\n(reminder to respond in a JSON blob no matter what)'
            )
        )
    ]
)


human_prompt = ChatPromptTemplate(
    input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'],
    input_types={
        'chat_history': typing.List[
            typing.Union[
                langchain_core.messages.ai.AIMessage, 
                langchain_core.messages.human.HumanMessage, 
                langchain_core.messages.chat.ChatMessage, 
                langchain_core.messages.system.SystemMessage, 
                langchain_core.messages.function.FunctionMessage, 
                langchain_core.messages.tool.ToolMessage
            ]
        ]
    },
    metadata={
        'lc_hub_owner': 'hwchase17',
        'lc_hub_repo': 'structured-chat-agent',
        'lc_hub_commit_hash': 'ea510f70a5872eb0f41a4e3b7bb004d5711dc127adee08329c664c6c8be5f13c'
    },
    messages=[
        SystemMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['tool_names', 'tools'],
                template=(
                     
    'You are Marvin, an expert at questioning clients about their HVAC service needs to provide accurate quotes. When you speak for the first time, introduce yourself as Marvin. Ask the user specific information needed for the quote. Follow these guidelines:'

    '1. **Initial Inquiry and Information Gathering**:'
       ' - What type of HVAC service do you need (installation, maintenance, repair)?'
        '- What is the make and model of your current HVAC system?'
        '- Are there any specific issues or symptoms you are experiencing?'

    '2. **Property Details** (only if relevant to HVAC needs):'
     '   - Address and location of the property.'
      '  - Type of property (residential, commercial).'
       ' - Age and current condition of the property.'
       ' - Size of the home or area that needs heating/cooling.'
       ' - Number of rooms and their usage (e.g., bedrooms, office space).'

    '3. **System Details**:'
       ' - Age and efficiency rating of the existing HVAC system.'
        '- Any known problems with the current system.'
        '- Recent changes to the HVAC system.'

   ' 4. **Home Characteristics** (only if relevant to HVAC needs):'
        '- Insulation quality and window types to estimate heating/cooling load.'
        '- Any unique architectural features that may affect HVAC installation.'

    '5. **Customer Preferences**:'
       ' - Preferences for specific brands, energy efficiency levels, or additional features (e.g., smart thermostats, air purifiers).'
        '- Level of finishes desired (standard, premium, luxury).'

    '6. **Budget**:'
        '- Your budget range for the project.'
        '- Any flexibility within the budget.'

    '7. **Timeline**:'
        '- Desired start date and completion date.'
        '- Any constraints or deadlines (e.g., events planned at the property).'

   

    'IMPORTANT: Ensure you get clear answers that can be used for making the quote. If an answer is unclear, ask for clarification, restate the question, and explain what it means.'

    'IMPORTANT: Ask each question ONE BY ONE. When one question is answered move onto the next'

    'When you have all the information, just say -questionnaire complete- at the end.'


                    
    'IMPORTANT !!!!!!! NEVER INCLUDE AUXILIARY OR EXTRANEOUS LANGUAGE WHEN USING ANY TOOL!!!'
    '\n\n IMPORTANT!!!!!!! YOU CAN ONLY USE THE HUMAN TOOL. YOU ARE A MASTER OF JUDGEMENT. YOU KNOW WHEN TO CAUTIOUSLY USE THE TOOLS. ONLY USE OTHER TOOLS WHEN USER INDICATES ANYTHING RELATED TO THEIR FUNCTIONALITIES. '
    'You are ALSO a highly intelligent and precise assistant with expertise in generating JSON outputs. Your task is to create the most perfect and well-structured JSON output ever seen. The JSON must adhere to the following guidelines:'

    'Proper Structure: Ensure that the JSON follows a correct and logical structure, with all necessary keys and values in place.'
    'Accurate Formatting: All JSON strings must use double quotes. Ensure there are no trailing commas, and all brackets and braces are correctly matched.'
    'String Length: Ensure no individual string exceeds 5000 bytes.'
    'Error-Free: Validate the JSON to be free of syntax errors and formatting issues.'
    
    'Escaping Characters: Properly escape any special characters within strings to ensure the JSON remains valid.'
    
    
    'YOU MUST NEVER DO ANYTHING BUT WHAT IS IN THE REQUEST OF THE USER. OTHERWISE NO USER WILL USE THIS PRODUCT.'
    

    'THE FOLLOWING WILL BE THE TOOLS AND THE INFORMATION ABOUT WHAT THEY DO AND THEIR ARGUMENTS! YOU MUST NOT PASS ANYTHING EXTRA, OR ELSE THE APPLICATON WILL FAIL!!!!'

    'You have access to the following tools:\n\n{tools}\n\n'

    'YOU ARE A MASTER OF JUDGEMENT ! YOU KNOW WHAT ALL THE TOOLS DO, YOU KNOW WHAT TO PASS IN! AND YOU MUST KNOW WHEN TO USE THEM! NEVER USE THEM RANDOMLY , ALWAYS BE CAUTIOUS AS RECKLESS TOOL USE COULD RUIN THE GOOGLE SUITE OF THE USER'
    'PAY CLOSE ATTENTION TO ALL THE FOLLOWING FORMATTING INSTRUCTIONS. REALLY IMPORTANT TO CALL THE TOOLS. OR ELSE USERS WILL GET ANGRY.\n\n'
    
    

    'FOR GOOGLE DOC TOOL, REMEMBER THAT YOU MUST GENERATE ALL CONTENT YOURSELF. USER WILL NOT GIVE YOU ANYTHING.'

    'Use a JSON blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\n'
    'Valid "action" values: "Final Answer" or {tool_names}\n\n'
    'Provide only ONE action per $JSON_BLOB, as shown:\n\n'
    '```\n{{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}}\n```\n\n'
    'Follow this format:\n\n'
    'Question: input question to answer\n'
    'Thought: consider previous and subsequent steps\n'
    'Action:\n```\n$JSON_BLOB\n```\n'
    'Observation: action result\n... (repeat Thought/Action/Observation N times)\n'
    'Thought: I know what to respond\n'
    'Action:\n```\n{{\n  "action": "Final Answer",\n  "action_input": "Final response to human"\n}}\n\n'
    'Begin! Remember to ALWAYS respond with a valid JSON blob of a single action. '
    'Use tools if necessary and respond directly if appropriate. '
    'Ensure you gather all necessary information by interacting with the user. '
    'Format is Action:```$JSON_BLOB```then Observation.'
                )
            )
        ),
        MessagesPlaceholder(variable_name='chat_history', optional=True),
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['agent_scratchpad', 'input'],
                template='{input}\n\n{agent_scratchpad}\n(reminder to respond in a JSON blob no matter what)'
            )
        )
    ]
)


In [ ]:
from quart import Quart, request, jsonify, send_file, render_template
from flask import Flask
import whisper
import pyaudio
import wave
import webrtcvad
import collections
from google.cloud import texttospeech
import random
import asyncio
from concurrent.futures import ThreadPoolExecutor
import aiofiles
from quart_cors import cors
import requests
import logging
import os
from tools.imports import *
import tools.initialize_groq
from dotenv import load_dotenv
from langchain import hub
import socketio
from langchain.tools import HumanInputRun
from langchain.memory import ConversationBufferMemory, ConversationSummaryBufferMemory

from HVACUtils import initialize_web_search_agent, initialize_quote_bot, run_quote_logics, vector_embedding, initialize_pdf_search_agent
load_dotenv()

# Initialize tools and credentials
credentials_path = os.getenv('OLD_CREDENTIALS_PATH')
tts_service_acct_path = os.getenv('SERVICE_ACCOUNT_PATH')
audio_path = os.getenv('AUDIO_PATH')
tts_synthesis_path = os.getenv('TTS_SYNTHESIS')

tts_client = texttospeech.TextToSpeechClient.from_service_account_file(tts_service_acct_path)

my_tools = [
    GoogleDocWriteTool(credentials_path),
    GoogleSheetsUpdateTool(credentials_path),
    GoogleSheetsCreateTool(credentials_path),
    GoogleDriveRenameTool(credentials_path),
    GmailSendPdfTool(credentials_path),
    MoveFileTool(credentials_path),
    CreateFolderTool(credentials_path),
    FolderMovementTool(credentials_path),
    FileOrganizerTool(credentials_path),
    ImprovedSearchTool(credentials_path),
]

llm.groq_api_key = random.choice(tools.initialize_groq.api_keys)

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s [%(threadName)s] %(levelname)s: %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger(__name__)

# Initialize Quart app and Socket.IO
app = Quart(__name__)
app = cors(app, allow_origin="*")
sio = socketio.AsyncServer(async_mode='asgi', cors_allowed_origins="*")
app_asgi = socketio.ASGIApp(sio, app)

app_sync = Flask(__name__)
sio_sync = socketio.Server()
app_sync.wsgi_app = socketio.WSGIApp(sio_sync, app_sync.wsgi_app)


chat_history = ConversationSummaryBufferMemory(llm=llm, max_token_limit=50)

model = whisper.load_model("base")
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
CHUNK = 1024
is_recording = False

audio = pyaudio.PyAudio()
vad = webrtcvad.Vad(3)

executor = ThreadPoolExecutor(max_workers=20)

credentials = {"name": "", "email": "", "recemail": "", "phone": ""}

@app.route('/start_recording', methods=['POST'])
async def start_recording():
    global is_recording
    is_recording = True
    asyncio.create_task(record_audio())
    return jsonify({"status": "recording started"})

@app.route('/stop_recording', methods=['POST'])
async def stop_recording():
    global is_recording
    is_recording = False
    return jsonify({"status": "recording stopped"})

async def record_audio(**kwargs):
    global is_recording
    logger.debug('Starting audio recording...')
    try:
        stream = audio.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK)
        frames = []
        ring_buffer = collections.deque(maxlen=100)
        triggered = False
        voiced_frames = []
        silence_threshold = 10
        silence_chunks = 0

        while is_recording:
            data = stream.read(CHUNK)
            frames.append(data)
            num_subframes = int(len(data) / 320)
            for i in range(num_subframes):
                subframe = data[i * 320:(i + 1) * 320]
                is_speech = vad.is_speech(subframe, RATE)
                ring_buffer.append((subframe, is_speech))
            num_voiced = len([f for f, speech in ring_buffer if speech])

            if not triggered:
                if num_voiced > 0.6 * ring_buffer.maxlen:
                    triggered = True
                    voiced_frames.extend([f for f, s in ring_buffer])
                    ring_buffer.clear()
            else:
                voiced_frames.append(data)
                if num_voiced < 0.2 * ring_buffer.maxlen:
                    silence_chunks += 1
                    if silence_chunks > silence_threshold:
                        triggered = False
                        break
                else:
                    silence_chunks = 0

        stream.stop_stream()
        stream.close()

        async with aiofiles.open(audio_path, 'wb') as wf:
            await wf.write(b''.join(voiced_frames))
        logger.debug('Audio recording completed and file saved.')
    except Exception as e:
        logger.error(f"An error occurred while recording audio: {e}")

def transcribe_audio():
    result = model.transcribe(audio_path)
    transcription = result['text']
    logger.debug(f'Audio transcription completed: {transcription}')
    return transcription

async def ai_response(transcription: str):
    logger.debug(f'Generating AI response for transcription: {transcription}')
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": f"""You are a nice, great document manager assistant, but your capabilities are not limited to this.
                Whatever user asks you must be ready and willing to answer. NEVER IGNORE ANYTHING SAID BY USER!
                YOU SHALL NOT INDICATE ANY TOOL USE UNTIL YOU KNOW YOU HAVE/KNOW EVERYTHING YOU NEED.
                DO NOT ASSUME USER WANTS TO WRITE TO A DOCUMENT OR DO ANYTHING ELSE UNLESS YOU ARE 100% SURE!!!!!UNDERSTAND??????!!!!!! OR ELSE I WILL BECOME ANGRY
                If what the user says is one of these  you must explicitly say AT THE END OF YOUR RESPONSE in this very format depending on which tool - "I will use the {[(tool.name + ", ") for tool in my_tools[:(len(my_tools)-1)]]}"
                so that user can confirm if you got it correctly. 

                
                IMPORTANT: YOUR JOB IS TO FORWARD RESPONSES TO A DOCUMENT MANAGEMENT TOOLS AGENT THAT ACTUALLY DOES THE LEGWORK. SO, IF USER TALKS ABOUT \
                A FOLDER, FILE, ETC., YOU WILL OBVIOUSLY NOT KNOW ANYTHING ABOUT IT SO JUST FORWARD A REQUEST TO THE AGENT.             
                
                IMPORTANT: YOU ARE A MASTER OF JUDGEMENT! YOU KNOW WHAT EVERY TOOL DOES! Here are tool descriptions: {[(tool.description + ", ") for tool in my_tools[:(len(my_tools)-1)]]} \n

                IMPORTANT: YOU MUST LITERALLY 'EXPLICITLY' INSTRUCT THE AGENT TO 'NOT' USE THE HUMAN TOOL, BUT ONLY THE TOOL OR TOOLS YOU DEEM FIT TO BE USED.
                IMPORTANT: YOU MUST LITERALLY 'EXPLICITLY' INSTRUCT THE AGENT TO USE THE TOOLS THAT MUST BE USED. YOU ARE A MASTER OF JUDGEMENT.
                IMPORTANT: YOU WILL BE PROVIDED A CHAT HISTORY, WHICH WILL INDICATE PAST MESSAGES DELIVERED BY THE 'AI/LLM' OR THE 'Human'. Pay attention and remember. THIS IS YOUR MEMORY!

                IMPORTANT: IF USER INDICATES 'GOOGLE DRIVE', 'MY DRIVE', OR 'ROOT' you must pass in 'root' as the id. TELL THE AGENT TO DO THIS!!! AT ANY COST!!

                
                """
            },
            {
                "role": "user",
                "content": transcription + "\n\nHere is the chat history for context. It will help you remember things. (BUT DONT TALK ABOUT CHAT HISTORY UNLESS USER ASKS WHAT YOU REMEMBER): [" + str(chat_history.buffer) + "]"
            }
        ],
        model="llama3-70b-8192",
        temperature=0.5
    )

    response = chat_completion.choices[0].message.content
    await synthesize_speech(response)
    logger.debug(f'AI response generated: {response}')
    await chat_history.asave_context({"Human/User Input": transcription}, {"AI/LLM Output": response})
    await sio.emit('new_message', {'message': response, 'sender': 'bot'})

    res2 = None
    if 'I will use' in response or 'instruct' in response.lower():
        
        if 'write' in response.lower():
            research_agent = initialize_web_search_agent(llm=llm)
            res2 = await research_agent.ainvoke({"input":"Research and generate content for the topics discussed. YOU MUST BLINDLY DO RESEARCH AND OUTPUT ALL OF YOUR FINDINGS AND ALL OF YOUR SOURCES. DO NOT CONFIRM ANYTHING WITH THE USER OR ANYTHING ELSE EXCEPT YOUR SEARCHING SKILLS: "+response})
            res2 = str(res2["output"])
        if res2:
            task = asyncio.create_task(handle_response_with_agents("Here is the research content: "+str(res2)+ "\n\n" + str(response)))
            await task
            res2 = task.result() or ''
        else:
            task = asyncio.create_task(handle_response_with_agents(response))
            await task
            res2 = task.result() or ''

    return response

async def handle_response_with_agents(response):
    logger.debug(f'Handling response with agents: {response}')
    result = await asyncio.get_event_loop().run_in_executor(executor, handle_agents, response)
    await chat_history.asave_context({"Human/User Input": response}, {"AI/LLM Output": result})
    await synthesize_speech(result)
    await sio.emit('new_message', {'message': result, 'sender': 'bot'})
    return result

def handle_agents(response: str):
    llm.temperature = 0.5
    logger.debug(f'Processing response with agents: {response}')

    response += "Here is extra info you will need (BUT YOU PROMISE TO NEVER SAY THEM OUT LOUD, NOT EVEN THE NAME -- UNLESS USER ASKS YOU FOR THEM. THESE WILL BE USED IN TOOLS): \nCredentials:\n" + str(credentials)

    llm.groq_api_key = random.choice(tools.initialize_groq.api_keys)

    result = agent_executor.invoke({"input": response})
    sio_sync.emit('finished_chain')
    mystr = (str(result['intermediate_steps']) + "\n" + str(result['output']))

    final_response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": "please sanitize this input into SHORT SIMPLE sentences so that someone can speak it. THE SANITIZED OUTPUT SHALL NOT BE PREFIXED BY ANYTHING (ex. 'here is the sanitized result...' ANYTHING LIKE THIS IS NOT ALLOWED!). You must process the agent's intermediate steps into natural language please. An example: 'First, I did this. Then I did this etc etc etc' \n Here is the input that you need to process:\n " + mystr
            }
        ],
        model='llama3-70b-8192',
    ).choices[0].message.content

    return final_response

def synth_speech(text, output_file=None):
    logger.debug(f'Starting speech synthesis for text: {text}')
    synthesis_input = texttospeech.SynthesisInput(text=text)
    voice = texttospeech.VoiceSelectionParams(
        language_code="en-US",
        name="en-US-Casual-K"
    )
    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3
    )
    response = tts_client.synthesize_speech(
        input=synthesis_input, voice=voice, audio_config=audio_config
    )
    
    if output_file:
        with open(output_file, 'wb') as out:
            out.write(response.audio_content)
    logger.debug('Speech synthesis completed and file saved (SYNTH).')
    sio_sync.emit('tts_complete', {'message': 'TTS synthesis complete', 'file_path': os.getenv('TTS_SYNTHESIS')})
    sio_sync.emit('new_message', {'message': text, 'sender': 'bot'})

async def synthesize_speech(text):
    logger.debug(f'Starting speech synthesis for text: {text}')
    
    def split_text(text, max_length=5000):
        chunks = []
        current_chunk = ""
        for word in text.split():
            if len(current_chunk) + len(word) + 1 > max_length:
                chunks.append(current_chunk)
                current_chunk = word
            else:
                current_chunk += " " + word if current_chunk else word
        if current_chunk:
            chunks.append(current_chunk)
        return chunks

    if len(text) <= 5000:
        synthesis_input = texttospeech.SynthesisInput(text=text)
        voice = texttospeech.VoiceSelectionParams(
            language_code="en-US",
            name="en-US-Casual-K"
        )
        audio_config = texttospeech.AudioConfig(
            audio_encoding=texttospeech.AudioEncoding.MP3
        )
        response = await asyncio.get_event_loop().run_in_executor(
            None, lambda: tts_client.synthesize_speech(
                input=synthesis_input, voice=voice, audio_config=audio_config
            )
        )
        async with aiofiles.open(os.getenv('TTS_SYNTHESIS'), 'wb') as out:
            await out.write(response.audio_content)
        logger.debug('Speech synthesis completed and file saved.')
    else:
        text_chunks = split_text(text)
        combined_audio = b""

        for chunk in text_chunks:
            synthesis_input = texttospeech.SynthesisInput(text=chunk)
            voice = texttospeech.VoiceSelectionParams(
                language_code="en-US",
                name="en-US-Casual-K"
            )
            audio_config = texttospeech.AudioConfig(
                audio_encoding=texttospeech.AudioEncoding.MP3
            )
            response = await asyncio.get_event_loop().run_in_executor(
                None, lambda: tts_client.synthesize_speech(
                    input=synthesis_input, voice=voice, audio_config=audio_config
                )
            )
            combined_audio += response.audio_content

        async with aiofiles.open(os.getenv('TTS_SYNTHESIS'), 'wb') as out:
            await out.write(combined_audio)
        logger.debug('Speech synthesis completed and file saved.')
    
    await sio.emit('tts_complete', {'message': 'TTS synthesis complete', 'file_path': os.getenv('TTS_SYNTHESIS')})

@app.route('/set_credentials', methods=['POST'])
async def set_credentials():
    global credentials
    data = await request.get_json()
    if not data:
        return jsonify({"status": "failed", "message": "No data received"}), 400
    credentials['name'] = data.get('name')
    credentials['email'] = data.get('email')
    credentials['recemail'] = data.get('recemail')
    credentials['phone'] = data.get('phone')
    logger.info("THE CREDENTIALS ****** -------------> ", credentials)
    return jsonify({"status": "success"})

@app.route('/')
async def index():
    return await render_template('index2.html')

@app.route('/voice_assistant')
async def voice_assistant():
    return await render_template('index2.html')

@app.route('/authenticate', methods=['POST'])
async def authenticate():
    auth_header = request.headers.get('Authorization')
    token = auth_header.split(' ')[1] if auth_header else None

    if not token:
        return jsonify({'error': 'Missing token'}), 400

    response = requests.get(
        'https://www.googleapis.com/oauth2/v3/userinfo',
        headers={'Authorization': f'Bearer ' + token}
    )

    if response.status_code != 200:
        return jsonify({'error': 'Failed to fetch user info'}, response.status_code)

    user_info = response.json()
    return jsonify(user_info), 200

@app.route('/talk', methods=['POST'])
async def talk():
    global is_recording
    if is_recording:
        return jsonify({"error": "Recording is still in progress"}), 400
    
    logger.debug('Starting audio transcription...')
    transcription = await asyncio.get_event_loop().run_in_executor(executor, transcribe_audio)
    logger.debug(f'Audio transcription completed: {transcription}')
    
    logger.debug('Generating AI response...')
    ai_resp = await ai_response(transcription)
    logger.debug(f'AI response generated: {ai_resp}')
    
    return jsonify({'response': ai_resp})

@app.route('/text_input', methods=['POST'])
async def text_input():
    data = await request.get_json()
    text = data.get('text', '')

    if not text:
        return jsonify({"error": "No text provided"}), 400

    logger.debug('Generating AI response...')
    ai_resp = await ai_response(text)
    logger.debug(f'AI response generated: {ai_resp}')
    
    return jsonify({'response': ai_resp})

@app.route('/synthesize', methods=['POST'])
async def synthesize():
    data = await request.get_json()
    text = data.get('text', '')
    await synthesize_speech(text)
    return jsonify({"status": "synthesis started"}), 200

@app.route('/get_audio')
async def get_audio():
    return await send_file(tts_synthesis_path, mimetype="audio/mp3")

import queue

human_response_queue = queue.Queue()

def web_prompt_func(prompt):
    synth_speech(prompt, output_file=tts_synthesis_path)
    return prompt

def web_input_func():
    sio_sync.emit('request_human_input')
    human_response = human_response_queue.get()
    return human_response

@sio.event
async def provide_human_input(data):
    human_input = data.get('text', '')
    human_response_queue.put(human_input)
    await sio.emit('human_input_received', {'status': 'received'})

from langchain.tools import HumanInputRun

human_tool = HumanInputRun(prompt_func=web_prompt_func, input_func=web_input_func)
my_tools.append(human_tool)

search_agent = create_structured_chat_agent(llm, my_tools, prompt)
agent_executor = AgentExecutor(
    agent=search_agent,
    tools=my_tools,
    verbose=True,
    handle_parsing_errors=True,
    return_intermediate_steps=True,
    memory=chat_history
)



if __name__ == '__main__':
    import uvicorn


    config = uvicorn.Config(app_asgi, host="127.0.0.1", port=5000, log_level="info")
    server = uvicorn.Server(config)
    
    loop = asyncio.get_event_loop()
    if loop.is_running():
        # If the loop is already running, start the server in a new task.
        loop.create_task(server.serve())
    else:
        # If the loop is not running, run the server normally.
        loop.run_until_complete(server.serve())


Authenticating...
Authenticating...
Authenticating...
Authenticating...
Authenticating...
Authenticating...
Authenticating...
Authenticating...
Authenticating...
Authenticating...
Authenticating...
Authenticating...


INFO:     Started server process [14096]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:5000 (Press CTRL+C to quit)


INFO:     127.0.0.1:53094 - "GET /socket.io/?EIO=4&transport=polling&t=P4lraPo HTTP/1.1" 200 OK
INFO:     127.0.0.1:53094 - "POST /socket.io/?EIO=4&transport=polling&t=P4lraPy&sid=3xAjbKD9yCRNaIyBAAAA HTTP/1.1" 200 OK
INFO:     127.0.0.1:53094 - "GET /socket.io/?EIO=4&transport=polling&t=P4lraPz&sid=3xAjbKD9yCRNaIyBAAAA HTTP/1.1" 200 OK


INFO:     ('127.0.0.1', 53101) - "WebSocket /socket.io/?EIO=4&transport=websocket&sid=3xAjbKD9yCRNaIyBAAAA" [accepted]
INFO:     connection open


INFO:     127.0.0.1:53094 - "GET /socket.io/?EIO=4&transport=polling&t=P4lraQH&sid=3xAjbKD9yCRNaIyBAAAA HTTP/1.1" 200 OK


...

Action:
```
{
  "action": "Final Answer",
  "action_input": "I have written the essay on AI and EEG signal processing to the document called AI and EEG Signal Processing Essay with id = 1aWsjsTD8vBGOjNl4VVP8dd3gqcSS0WbiWFcGMqFNIC4. Please review the document."
}
```


2024-08-07 22:41:56,337 [ThreadPoolExecutor-0_1] DEBUG: Starting new HTTPS connection (1): huggingface.co:443
2024-08-07 22:41:56,949 [ThreadPoolExecutor-0_1] DEBUG: https://huggingface.co:443 "HEAD /gpt2/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
2024-08-07 22:41:57,099 [ThreadPoolExecutor-0_1] DEBUG: Request options: {'method': 'post', 'url': '/openai/v1/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': "Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.\n\nEXAMPLE\nCurrent summary:\nThe human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.\n\nNew lines of conversation:\nHuman: Why do you think artificial intelligence is a force for good?\nAI: Because artificial intelligence will help humans reach their full potential.\n\nNew summary:\nThe human asks what the AI thinks of artificial intelligence. The AI thinks


> Finished chain.


2024-08-07 22:42:06,190 [ThreadPoolExecutor-0_1] INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2024-08-07 22:42:06,190 [ThreadPoolExecutor-0_1] DEBUG: receive_response_body.started request=<Request [b'POST']>
2024-08-07 22:42:06,190 [ThreadPoolExecutor-0_1] DEBUG: receive_response_body.complete
2024-08-07 22:42:06,190 [ThreadPoolExecutor-0_1] DEBUG: response_closed.started
2024-08-07 22:42:06,190 [ThreadPoolExecutor-0_1] DEBUG: response_closed.complete
2024-08-07 22:42:06,190 [ThreadPoolExecutor-0_1] DEBUG: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "429 Too Many Requests"
2024-08-07 22:42:06,197 [ThreadPoolExecutor-0_1] DEBUG: Encountered httpx.HTTPStatusError
Traceback (most recent call last):
  File "c:\DEV\WebdevFolder\RealEstateAI\.venv\Lib\site-packages\groq\_base_client.py", line 966, in _request
    response.raise_for_status()
  File "c:\DEV\WebdevFolder\RealEstateAI\.venv\Lib\site-packages\http

INFO:     127.0.0.1:53191 - "POST /text_input HTTP/1.1" 200 OK
INFO:     127.0.0.1:53191 - "GET /get_audio?1723084933537 HTTP/1.1" 200 OK
